In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame. 
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2020-04-18 23:06:02,-46.19,168.86,53.01,74,80,8.01
1,1,Fomboni,KM,2020-04-18 23:06:03,-12.28,43.74,82.76,72,75,10.85
2,2,Nouakchott,MR,2020-04-18 23:02:02,18.09,-15.98,66.20,88,0,14.99
3,3,Houma,US,2020-04-18 23:06:03,29.60,-90.72,72.00,94,40,5.75
4,4,Albany,US,2020-04-18 23:02:35,42.60,-73.97,46.00,55,76,11.52


In [3]:
# Get the data types 
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key 
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))



In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temps = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Fomboni,KM,2020-04-18 23:06:03,-12.28,43.74,82.76,72,75,10.85
13,13,Souillac,MU,2020-04-18 23:02:46,-20.52,57.52,78.01,78,75,12.75
26,26,Avera,US,2020-04-18 22:54:53,33.19,-82.53,75.20,27,1,10.29
29,29,Puerto Ayora,EC,2020-04-18 22:56:29,-0.74,-90.35,84.20,70,40,10.29
34,34,Yarada,IN,2020-04-18 23:06:10,17.65,83.27,80.85,80,0,11.34
36,36,Mabaruma,GY,2020-04-18 23:06:10,8.20,-59.78,77.05,86,30,7.25
37,37,Dwarka,IN,2020-04-18 23:01:52,22.24,68.97,79.92,82,22,12.93
41,41,Hilo,US,2020-04-18 22:52:54,19.73,-155.09,82.40,54,20,12.75
42,42,Rikitea,PF,2020-04-18 23:00:12,-23.12,-134.97,77.41,70,42,5.93
43,43,Petatlan,MX,2020-04-18 23:06:12,17.52,-101.27,87.80,55,5,9.17


In [10]:
preferred_cities_df.count()


City_ID       180
City          180
Country       180
Date          180
Lat           180
Lng           180
Max Temp      180
Humidity      180
Cloudiness    180
Wind Speed    180
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Fomboni,KM,82.76,-12.28,43.74,
13,Souillac,MU,78.01,-20.52,57.52,
26,Avera,US,75.20,33.19,-82.53,
29,Puerto Ayora,EC,84.20,-0.74,-90.35,
34,Yarada,IN,80.85,17.65,83.27,
...,...,...,...,...,...,...
559,Lazaro Cardenas,MX,81.37,17.96,-102.20,
562,Ipixuna,BR,78.80,-1.76,-48.80,
566,El Tarra,CO,76.82,8.58,-73.09,
569,Tucuman,AR,75.20,-26.82,-65.22,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [20]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Fomboni,KM,82.76,-12.28,43.74,Auberge les abou
13,Souillac,MU,78.01,-20.52,57.52,Bils' Villa
26,Avera,US,75.20,33.19,-82.53,
29,Puerto Ayora,EC,84.20,-0.74,-90.35,Finch Bay Galapagos Hotel
34,Yarada,IN,80.85,17.65,83.27,"D-26/2, Dolphin Hill,"
...,...,...,...,...,...,...
559,Lazaro Cardenas,MX,81.37,17.96,-102.20,Hotel Quinta Antigua
562,Ipixuna,BR,78.80,-1.76,-48.80,Casa da Oscarina
566,El Tarra,CO,76.82,8.58,-73.09,TOS.corp
569,Tucuman,AR,75.20,-26.82,-65.22,Dallas Hotel Tucuman


In [21]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [23]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [24]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))